In [2]:
import subprocess
import shlex

def run_command(command):
    print("Launch " + command)
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [3]:
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt 
import numpy as np 
def get_df_process(host):
    host="vm-vmonly-db-from1"
    port=5432
    login="bonitauser"
    password="migrationTestDb_18"
    dbname="bonitadb"
    conn = psycopg2.connect("host = '{0}' dbname = '{1}' user = '{2}' password = '{3}' port = {4}".
             format(host, dbname, login, password, port))
    sql = "select api_id.sourceobjectid as id, api_begin.startdate as start, api_state1.startdate as ready, api_end.enddate as end\n"\
    ", afi_begin.sourceobjectid as task_id, afi_begin.reachedstatedate as task_start\n"\
    ", afi_assign.reachedstatedate as task_ready, afi_assign.claimeddate as task_assign, afi_end.reachedstatedate as task_end\n"\
    "from (\n"\
    "	select distinct sourceobjectid\n"\
    "	from arch_process_instance api) as api_id\n"\
    "join arch_process_instance api_begin on api_id.sourceobjectid = api_begin.sourceobjectid and api_begin.stateid = 0\n"\
    "left join arch_process_instance api_end on api_id.sourceobjectid = api_end.sourceobjectid and api_end.stateid = 6\n"\
    "left join arch_process_instance api_state1 on api_id.sourceobjectid = api_state1.sourceobjectid and api_state1.stateid = 1\n"\
    "left join arch_flownode_instance afi_begin on api_id.sourceobjectid = afi_begin.parentcontainerid and afi_begin.stateid = 32 and afi_begin.kind = 'user'\n"\
    "left join arch_flownode_instance afi_end on afi_begin.sourceobjectid = afi_end.sourceobjectid and afi_end.stateid = 2 and afi_end.kind = 'user'\n"\
    "left join arch_flownode_instance afi_assign on afi_begin.sourceobjectid = afi_assign.sourceobjectid and afi_assign.stateid = 4 and afi_assign.kind = 'user' \n"\
    "order by id,task_id"
    cur = conn.cursor()
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns=["id", "start", "ready", "end", "task_id", "task_start", "task_ready", "task_assign", "task_end"])
    df["duration"] = df["end"] - df["start"]
    df["start"] = pd.to_datetime(df["start"]/1000, unit="s")
    df["end"] = pd.to_datetime(df["end"]/1000, unit="s")

    return df

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# initialize infra
command = './create_vm.sh'
res = run_command(command)

In [4]:
# 

now=datetime.datetime.now(tzlocal())
strnow = now.strftime("%Y%m%d%H%M%S")
migration_result_name = "xp2_migr_{0}".format(strnow)
path = "./results/{0}/".format(migration_result_name)
import pathlib
pathlib.Path(path).mkdir(parents=True, exist_ok=True) 
print ("Migration result name : " + migration_result_name)
with cd("./test_executor/faban/master/bin"):
    run_command("./shutdown.sh")
    run_command("./startup.sh")


Migration result name : xp2_migr_20180623151601
Launch ./shutdown.sh
Launch ./startup.sh
Starting Faban Server
Please point your browser to http://127.0.0.1:9980/


### Initialize swarm on infrastructure 

In [5]:
# initialize Swarm on infra & upload files
import csv
#generate agent conf
agent_conf_file = "config.txt"
agent_conf={}
agent_conf["tenant_name"] = "tenant1"
agent_conf["login"] = "user1"
agent_conf["pwd"] = "user1"
agent_conf["nb_agent"] = 50
agent_conf["nb_process"] = 50

with open(path+agent_conf_file, 'w') as csvfile:
    row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], agent_conf["nb_agent"], agent_conf["nb_process"]]
    wr = csv.writer(csvfile, delimiter=';')
    wr.writerow(row)
with open("./config.txt", 'w') as csvfile:
    row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], agent_conf["nb_agent"], agent_conf["nb_process"]]
    wr = csv.writer(csvfile, delimiter=';')
    wr.writerow(row)

command = './init_swarm.sh'
res = run_command(command)

if (res != 0):
    print("Fail.")



Launch ./init_swarm.sh
[ERROR]: No handlers could be found for logger
"msrestazure.azure_active_directory"
[DEPRECATION WARNING]: 'include' for playbook includes. You should use
'import_playbook' instead. This feature will be removed in version 2.8.
Deprecation warnings can be disabled by setting deprecation_warnings=False in
ansible.cfg.

PLAY [vmonly] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-db-to1]

TASK [include_vars] ************************************************************
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-db-to1]

TASK [docker : Run the equivalent of "apt-get update".] ************************
changed: [vm-vmonly-db-ref]
cha

changed: [localhost]

TASK [swarm-manager : get docker swarm manager token] **************************
changed: [localhost]

TASK [swarm-manager : get docker swarm worker token] ***************************
changed: [localhost]

TASK [swarm-manager : debug] ***************************************************
ok: [localhost] => {
"ansible_default_ipv4.address": "10.0.0.5"
}

TASK [swarm-manager : set_fact] ************************************************
ok: [localhost]

TASK [swarm-manager : create registry] *****************************************
changed: [localhost]

PLAY [vmonly] ******************************************************************

TASK [swarm-worker : get docker info] ******************************************
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-db-to1]

TASK [swarm-worker : quit swarm cluster] ***************************************
changed: [vm-vmonly-bpm-agen

changed: [localhost] => (item=vm-vmonly-bpm-agent1)
changed: [localhost] => (item=vm-vmonly-bpms-from1)
changed: [localhost] => (item=vm-vmonly-bpms-to1)
changed: [localhost] => (item=vm-vmonly-db-from1)
changed: [localhost] => (item=vm-vmonly-db-ref)
changed: [localhost] => (item=vm-vmonly-db-to1)

PLAY RECAP *********************************************************************
localhost                  : ok=13   changed=8    unreachable=0    failed=0
vm-vmonly-bpm-agent1       : ok=26   changed=8    unreachable=0    failed=0
vm-vmonly-bpms-from1       : ok=39   changed=11   unreachable=0    failed=0
vm-vmonly-bpms-to1         : ok=39   changed=11   unreachable=0    failed=0
vm-vmonly-db-from1         : ok=21   changed=6    unreachable=0    failed=0
vm-vmonly-db-ref           : ok=21   changed=6    unreachable=0    failed=0
vm-vmonly-db-to1           : ok=21   changed=6    unreachable=0    failed=0



### Initialize result name, Start Faban and reset Docker

In [18]:



run_command("./reset_docker.sh")
    

Launch ./reset_docker.sh
"docker stack rm" requires at least 1 argument.
See 'docker stack rm --help'.

Usage:  docker stack rm STACK [STACK...] [flags]

Remove one or more stacks


1

### Provision and test for various numbers of processes

In [ ]:
# provision and test
import time

dur = 30

for i in range(1,7):
    file = open(path+"{0}_migr_{1}.csv".format(migration_result_name,i), "w")
    file.write("ts,nb,duration_disable,duration_move,duration_enable,duration_total\n")
    file.close()
    faban_agent = i
    
    #run_command("./launch_agents.sh http://vm-vmonly-bpms-from1:8080/bonita agent-pre")
    tries = 3
    while tries >= 0:
        run_command("./reset_docker.sh")
        
        with cd ("./test_executor/templates/deployment_descriptor/template"):
            res = run_command("docker stack deploy --compose-file=docker-compose.yml {0}".format(migration_result_name))
            if (res != 0):
                tries = tries - 1
                time.sleep(10) 
                continue
            else:
                time.sleep(120) 
                break

    with cd ("./test_executor"):
        run_command("./benchflow_onlytest.sh generate_test {0} TestHumanTask-1.0.bar 5 {1} 0 tenant1".format(faban_agent, dur))

        run_command("./benchflow_onlytest.sh execute_tests {0}".format(migration_result_name))
    
    result_agent_filename = "{0}_agent{1}_{2}.csv".format(migration_result_name, "pre", i)
    run_command("ssh ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./{0}".format(result_agent_filename))
    run_command("scp ubuntu@vm-vmonly-bpm-agent1:{0} {1}".format(result_agent_filename, path+result_agent_filename))
    
    run_command("docker service rm {0}_bpm-agent".format(migration_result_name))
    res1 = run_command("./migrate.sh tenant1 {0}_migr_{1}".format(path+migration_result_name, i))
    if (res1 !=0):
        print("Error in first migration.. skipping")
    df_pre = get_df_process("vm-vmonly-db-from1")
    #df_pre.plot(x="start", y="duration", style=".")

    date_postmigr = datetime.datetime.now()

    #run_command("./launch_agents.sh http://vm-vmonly-bpms-to1:8081/bonita agent-post")
    with cd ("./test_executor"):
        run_command("./benchflow_onlytest.sh generate_test {0} TestHumanTask-1.0.bar 5 {1} 0 tenant1 config2.properties".format(faban_agent, dur))

        run_command("./benchflow_onlytest.sh execute_tests {0} config2.properties".format(migration_result_name))

    result_agent_filename = "{0}_agent{1}_{2}.csv".format(migration_result_name, "post", i)
    run_command("ssh ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./{0}".format(result_agent_filename))
    run_command("scp ubuntu@vm-vmonly-bpm-agent1:{0} {1}".format(result_agent_filename, path+result_agent_filename))
        
    df = get_df_process("vm-vmonly-db-from1")

    df_pre = df[df["start"] < date_postmigr]
    df_post = df[df["start"] >= date_postmigr]

    df_pre.to_csv(path+"{0}_pre_{1}.csv".format(migration_result_name ,i))
    df_post.to_csv(path+"{0}_post_{1}.csv".format(migration_result_name ,i))    

    df_pre["start"] = df_pre["start"] - min(df_pre["start"])
    df_post["start"] = df_post["start"] - min(df_post["start"])

    df_pre = df_pre.set_index(pd.DatetimeIndex(df_pre["start"]))   
    df_post=df_post.set_index(pd.DatetimeIndex(df_post["start"]))
    
    fig,ax = plt.subplots()
    df_pre.resample("1s", how="mean").plot(y="duration", style="ro",ax=ax)
    df_post.resample("1s", how="mean").plot( y="duration", style="bx",ax=ax)
    
    fig,ax = plt.subplots()
    df_pre.plot(y="duration", style="ro",ax=ax)
    df_post.plot( y="duration", style="bx",ax=ax)

Launch ./reset_docker.sh
"docker stack rm" requires at least 1 argument.
See 'docker stack rm --help'.

Usage:  docker stack rm STACK [STACK...] [flags]

Remove one or more stacks
Launch docker stack deploy --compose-file=docker-compose.yml xp2_migr_20180623151601
Ignoring unsupported options: restart

Creating network xp2_migr_20180623151601_default
Creating service xp2_migr_20180623151601_bonita-to
Creating service xp2_migr_20180623151601_start_dependencies
Creating service xp2_migr_20180623151601_db-ref
Creating service xp2_migr_20180623151601_db-from
Creating service xp2_migr_20180623151601_bonita-from
Creating service xp2_migr_20180623151601_db-to
Launch ./benchflow_onlytest.sh generate_test 1 TestHumanTask-1.0.bar 5 30 0 tenant1
The generated test ID is hDUn0L52U1W7CzAfvAlhhkCcQniPX0cq
Compiling the Faban driver
Buildfile: /home/ubuntu/scripts_gr/test_executor/drivers/hDUn0L52U1W7CzAfvAlhhkCcQniPX0cq/build.xml

init:
[mkdir] Created dir: /home/ubuntu/scripts_gr/test_executor/driv

In [90]:
result_agent_filename = "{0}_agent{1}_{2}.csv".format(migration_result_name, "post", i)
run_command("ssh ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps  -q --filter ancestor=bpmsagent):/Appagent/result.csv ./{0}".format(result_agent_filename))
run_command("scp ubuntu@vm-vmonly-bpm-agent1:{0} {1}".format(result_agent_filename, path+result_agent_filename))


Launch ssh ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps  -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp2_migr_20180621132541_agentpost_1.csv
Launch scp ubuntu@vm-vmonly-bpm-agent1:xp2_migr_20180621132541_agentpost_1.csv ./results/xp2_migr_20180621132541/xp2_migr_20180621132541_agentpost_1.csv


0